In [1]:
%load_ext autoreload
%autoreload 2 

### Check embedding error


In [ ]:
path = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra/word2vec/Advanced Micro Devices X86-64/aidra/81193e9a87778d7899a523adc7949f1a8af267d268e1dd51298165c22b890f4e.gpickle"

import networkx as nx
import pickle
import torch

with open(path, "rb") as f:
    G = pickle.load(f)
    
node = "0x4012f0L"

print(G.nodes[node])
G.nodes[node]["x"] = []
print(G.nodes[node])

# with open(path, "wb") as f:
#     pickle.dump(G, f)

In [ ]:
import torch
def check_model_weights(model_path, device):
    # 載入模型
    checkpoint = torch.load(model_path, map_location=device)
    # 1. 檢查state dict是否為空
    if not checkpoint["model_state_dict"]:
        print("Warning: Model state dict is empty!")
        return False
        
    # 2. 印出模型的所有權重名稱和形狀
    for name, param in checkpoint["model_state_dict"].items():
        print(f"Layer: {name} | Shape: {param.shape}")
        print(param)
        
    # 3. 檢查權重是否包含非零值
    for param in checkpoint["model_state_dict"].values():
        if torch.all(param == 0):
            print(f"Warning: Found all-zero parameter tensor!")
            
    return True

In [ ]:
path = "pretrained/x86_pretrained_20241122_1616/epoch_1342_best_backbone.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

check_model_weights(path, device)

### Check validation dataset & test dataset difference

In [ ]:
import pickle
testPath = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra/word2vec/testData.pkl"
valPath = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra/word2vec/valData.pkl"

with open(testPath, "rb") as f:
    test = pickle.load(f)
    
with open(valPath, "rb") as f:
    val = pickle.load(f)



In [ ]:
print(len(test[0]))
print(len(val[0]))

In [ ]:
testAvgLen = sum([len(graph.x) for graph in test[0]]) / len(test[0])
valAvgLen = sum([len(graph.x) for graph in val[0]]) / len(val[0])

print(testAvgLen)
print(valAvgLen)

### Get label dictionary data when loading data

In [ ]:
from utils import load_config
from loadDataset import LoadDataset
from trainModule import TestModule
import os

seeds = [6, 7, 10, 11, 19, 22, 31, 42, 666, 888]

for seed in seeds:
    configPath = f"/home/manying/Projects/fcgFewShot/config/config_NICT_Ghidra_x86_64_{seed}.json"
    options = load_config(configPath)
    
    options["paths"]["data"]["embedding_folder"] = "/home/manying/Projects/fcgFewShot/embeddings"
    dataset = LoadDataset(options)
    test = TestModule(configPath, dataset, options)


### Test: train on 10-way test on 5-way performance
Testing on seed_6_baseline: LP 10-way 5-shot

In [ ]:
from utils import load_config, save_config
import os
from loadDataset import LoadDataset
from trainModule import TestModule
configPath = "/home/mandy/Projects/few_shot_fcg/checkpoints/x86_64_withVal_withPretrain_ghidra_6_baseline/10way_5shot_LabelPropagation_alpha0.7_k20_20250315_155140/config.json"
options = load_config(configPath)
newConfigPath = os.path.join(os.path.dirname(configPath), "config_5way.json")
### change settings
options["settings"]["few_shot"]["test"]["class_per_iter"] = 5
options["settings"]["train"]["distance"] = "euclidean"
save_config(options, newConfigPath)

dataset = LoadDataset(options)

test = TestModule(newConfigPath, dataset)
test.eval()

### Analysis diff backbone parameters

In [ ]:
from models import GraphSAGELayer, GATLayer, GCNLayer, GINLayer


def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

dims = dict(dim_in=128, dim_h=128, dim_o=128, num_layers=3)
models = {
    'GCN'      : GCNLayer(**dims),
    'GraphSAGE': GraphSAGELayer(**dims),
    'GAT'      : GATLayer(**dims, heads=8),
    'GIN'      : GINLayer(**dims)
}

for name, m in models.items():
    print(f'{name}: {count_params(m):,} parameters')


### Check fcg node without "x"

In [ ]:
check_path = "/home/manying/Projects/fcgFewShot/dataset/data_ghidra_fcg_openset/Intel 80386"

import os, pickle

for familyFolder in os.listdir(check_path):
    familyPath = os.path.join(check_path, familyFolder)
    if not os.path.isdir(familyPath):
        continue
    for file in os.listdir(familyPath):
        filePath = os.path.join(familyPath, file)
        if not file.endswith(".gpickle"):
            continue
        with open(filePath, "rb") as f:
            data = pickle.load(f)
            for node in data.nodes:
                if "x" not in data.nodes[node]:
                    print(f"Node {node} in file {filePath} does not have 'x' attribute.")
                    continue

In [ ]:
file_path = "./dataset/data_ghidra_fcg/Intel 80386/ddostf/000f5bc23812367aecf93ff5d6c96ac644f0ae819096af6eab13eb1993b8dbe4.gpickle"

### Check ARM malware

In [ ]:
import networkx as nx
import pickle
import os

familyList = ["tediss", "dowgin", "mobidash", "helper", "sagent", "zergrush", "zhtrap", "rootnik", "boqx", "mirai", "gafgyt", "shixot", "feejar", "gluper", "dofloo", "dnsamp", "sidewalk", "wapron", "badpac", "ngioweb", "tekya", "monitorminor", "meterpreter"]
dataPath = "/home/manying/Projects/fcgFewShot/dataset/data_ghidra_fcg/ARM"
embedPath = "/home/manying/Projects/fcgFewShot/embeddings/arm_withVal_ghidra_42/word2vec/ARM"

for familyFolder in familyList:
    familyPath = os.path.join(dataPath, familyFolder)
    embedFamilyPath = os.path.join(embedPath, familyFolder)
    if not os.path.isdir(familyPath):
        print(f"Family folder {familyFolder} does not exist in {dataPath}.")
        continue
    for file in os.listdir(familyPath):
        filePath = os.path.join(familyPath, file)
        print(filePath)
        if not file.endswith(".gpickle"):
            continue
        with open(filePath, "rb") as f:
            data = pickle.load(f)
            print(data)
            for node in data.nodes:
                print(data.nodes[node])
    for file in os.listdir(embedFamilyPath):
        filePath = os.path.join(embedFamilyPath, file)
        if not file.endswith(".gpickle"):
            continue
        with open(filePath, "rb") as f:
            data = pickle.load(f)
            print(data)
            for node in data.nodes:
                print(data.nodes[node])

    break


### Test maml

In [2]:
from utils import load_config
from loadDataset import LoadDataset
from trainModule import TestModule, TrainModule
from fcgVectorize import FCGVectorize
import os
import warnings

path = "./config/config_maml_gcn.json"

options = load_config(path)

dataset = LoadDataset(options)


Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Loading openset data...
Openset data shape: (342, 16)


In [ ]:
trainModule = TrainModule(options, dataset)
trainModule.train()

Setting up the training module...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec...
Loading training data...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/trainData.pkl...


/home/mandy/Tools/anaconda3/envs/prototype/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Loading validation data...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/valData.pkl...
Device: cuda:0
Model: MAMLLoss(
  (encoder): GraphClassifier(
    (backbone): GCN(
      (gcn_convs): ModuleList(
        (0-2): 3 x GCNConv(128, 128)
      )
      (norms): ModuleList(
        (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (classifier): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=64, out_features=5, bias=True)
    )
  )
)
Loss function: MAMLLoss(
  (encoder): GraphClassifier(
    (backbone): GCN(
      (gcn_convs): ModuleList(
        (0-2): 3 x GCNConv(128, 128)
      )
      (norms): ModuleList(
        (0-2): 3 x BatchNorm1d(128, eps=1e-05, mo

Epoch 1/200 (Training): 100%|██████████| 100/100 [00:31<00:00,  3.18it/s, loss=1.3864, acc=0.4200, openset_auc=N/A]


Avg Train Loss: 1.0310, Avg Train Acc: 0.6364 (Best)


Epoch 1/200 (Validation): 100%|██████████| 100/100 [00:23<00:00,  4.27it/s, loss=1.5633, acc=0.2400, openset_auc=N/A]


Avg Val Loss: 1.3968, Avg Val Acc: 0.4266 (Best)
Current learning rate: [0.001]
Patience: 0/20


Epoch 2/200 (Training):  28%|██▊       | 28/100 [00:08<00:16,  4.49it/s, loss=0.9900, acc=0.6000, openset_auc=N/A]

In [ ]:
test = TestModule("/home/mandy/Projects/few_shot_fcg/checkpoints/x86_64_withVal_withPretrain_ghidra_6_others/5way_10shot_MAML_gcn_20250704_225322/config.json", dataset)
test.eval()


Setting up the testing module...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/testData.pkl...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec...
Generating open set testing data...
Loading openset data...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/opensetData_random_0.2.pkl...


/home/mandy/Tools/anaconda3/envs/prototype/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Finish setting up the testing module
Model path is not provided. Using the best model...
Record evaluation log...
Loading model from /home/mandy/Projects/few_shot_fcg/checkpoints/x86_64_withVal_withPretrain_ghidra_6_others/5way_10shot_MAML_gcn_20250704_225322/epoch_1_0.19999998807907104_best.pth


/home/mandy/Projects/few_shot_fcg/trainModule.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_location=self.devic

Model: MAMLLoss(
  (encoder): GraphClassifier(
    (backbone): GCN(
      (gcn_convs): ModuleList(
        (0-2): 3 x GCNConv(128, 128)
      )
      (norms): ModuleList(
        (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (classifier): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=64, out_features=5, bias=True)
    )
  )
)
Start evaluation... (testing dataset)
Open set evaluation enabled
Open set m_samples: 50
Epoch 1


...

targets: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4], device='cuda:0')
predictions: tensor([3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3,
        3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 2, 3,
        2, 3], device='cuda:0')


Epoch 1/10 (Testing):  34%|███▍      | 34/100 [00:09<00:16,  3.90it/s, loss=1.0898, acc=0.6000, openset_auc=0.3232]

targets: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 2, 0, 2, 2, 0,
        0, 0], device='cuda:0')


Epoch 1/10 (Testing):  35%|███▌      | 35/100 [00:09<00:16,  3.95it/s, loss=1.2890, acc=0.6000, openset_auc=0.3808]

targets: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 1, 0, 3, 0,
        0, 0], device='cuda:0')
targets: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 3,
        1, 3], device='cuda:0')
